In [1]:
import time
import requests
import pandas as pd

In [2]:
bearer_token="AAAAAAAAAAAAAAAAAAAAAAT4iAEAAAAAd07UAVq82fIOZm29Yfoqvf6qsMQ%3DlCeqKkZMtJh0mZynK5cyNYzZO70X0tfkvCaokbkyu3SYpJxB8x"
def bearer_oauth(r):
    """
    Method required by bearer token authentication.
    """

    r.headers["Authorization"] = f"Bearer {bearer_token}"
    r.headers["User-Agent"] = "v2FullArchiveSearchPython"
    return r

In [3]:
user_id = "1603715536013099009"
search_url = "https://api.twitter.com/2/tweets/{}?tweet.fields=public_metrics&expansions=attachments.media_keys&media.fields=public_metrics,non_public_metrics".format(user_id)
query_params={}

In [4]:
# Number of requests allowed per time window
RATE_LIMIT = 15
# Time window (in seconds)
TIME_WINDOW = 15

# Bucket for tracking remaining requests
bucket = RATE_LIMIT
# Timestamp of last bucket refill
last_refill = time.time()

def connect_to_endpoint(url, params):
    global bucket
    global last_refill
    # Check if bucket is empty
    if bucket <= 0:
        # Check if time window has expired
        current_time = time.time()
        if current_time - last_refill > TIME_WINDOW:
            # Refill bucket and update timestamp
            bucket = RATE_LIMIT
            last_refill = current_time
        else:
            # Wait for time window to expire
            time_to_wait = last_refill + TIME_WINDOW - current_time
            time.sleep(time_to_wait)
            bucket = RATE_LIMIT
    # Send request
    response = requests.request("GET", url, auth=bearer_oauth, params=params)
    print(response.status_code)
    if response.status_code != 200:
        raise Exception(response.status_code, response.text)
    # Decrement bucket
    bucket -= 1
    return response

In [5]:
followers = connect_to_endpoint(search_url, query_params)
print(followers.json())

200
{'data': {'id': '1603715536013099009', 'edit_history_tweet_ids': ['1603715536013099009'], 'public_metrics': {'retweet_count': 92, 'reply_count': 41, 'like_count': 624, 'quote_count': 3, 'impression_count': 44621}, 'text': 'Twitter has removed "Twitter for Iphone" and "Twitter for Android"\nIf you can still see it update your app.'}}
